In [5]:
! pip install reportlab matplotlib
import pandas as pd
import matplotlib.pyplot as plt
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import os

def create_bar_graph(data_file, image_path):
    """Creates a bar graph from a CSV file and saves it as an image."""
    try:
        df = pd.read_csv(data_file, delimiter='\t')

        if df.empty:
            print(f"Warning: No data found in {data_file}")
            return

        months = df['Month']
        science_scores = df['Science']
        arts_scores = df['Arts']

        x = range(len(months))
        width = 0.4

        fig, ax = plt.subplots(figsize=(14, 5))
        ax.bar([i - width / 2 for i in x], science_scores, width, label='Science', color='blue')
        ax.bar([i + width / 2 for i in x], arts_scores, width, label='Arts', color='red')

        ax.set_xlabel("Month")
        ax.set_ylabel("Scores")
        ax.set_title("Monthly Scores Comparison: Science vs Arts")
        ax.set_xticks(x)
        ax.set_xticklabels(months, rotation=45, ha='right')  # Improves label clarity
        ax.legend()

        plt.tight_layout()
        plt.savefig(image_path)
        plt.close()
    except FileNotFoundError:
        print(f"Error: File not found at {data_file}")
    except KeyError as e:
        print(f"Error: Column '{e.args[0]}' not found in the CSV file.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

def generate_report(filename, title, content, image_path):
    """Generates a PDF report using ReportLab."""
    try:
        c = canvas.Canvas(filename, pagesize=letter)
        width, height = letter

        # Title
        c.setFont("Helvetica-Bold", 16)
        c.drawCentredString(width / 2, height - 50, title)

        # Content
        c.setFont("Helvetica", 12)
        text = c.beginText(50, height - 100)
        text.setFont("Helvetica", 12)
        text.setLeading(14)

        for line in content.split('\n'):
            text.textLine(line)

        c.drawText(text)

        # Insert Bar Graph if it exists
        if os.path.exists(image_path):
            c.drawImage(image_path, 80, height - 400, width=400, height=300)
        else:
            print(f"Warning: Image file not found at {image_path}, skipping insertion.")

        c.showPage()
        c.save()
        print(f"Report saved as {filename}")
    except Exception as e:
        print(f"An unexpected error occurred while generating the report: {e}")

# Loop to generate reports
for i in range(1, 5):
    title = f"Student {i} Report"
    content = "This is an automatically generated student report."

    try:
        # Read Excel data
        file_path = 'Students Marks list.xlsx'
        sheet_name = f'Student {i}'

        if not os.path.exists(file_path):
            print(f"Error: Excel file '{file_path}' not found.")
            continue

        worksheet = pd.read_excel(file_path, sheet_name=sheet_name)

        # Save worksheet data as CSV file for processing
        data_file = f"data{i}.csv"
        worksheet.to_csv(data_file, sep='\t', index=False)

        # Generate bar graph
        image_path = f"bar_graph_{i}.png"
        create_bar_graph(data_file, image_path)

        # Generate report
        generate_report(f"report_{i}.pdf", title, content, image_path)
    except ValueError:
        print(f"Error: Sheet '{sheet_name}' not found in '{file_path}'.")
    except Exception as e:
        print(f"An unexpected error occurred in the main loop: {e}")


Report saved as report_1.pdf
Report saved as report_2.pdf
Report saved as report_3.pdf
Report saved as report_4.pdf
